In [1]:
import pickle
import random

In [2]:
import pandas
import pickle
import itertools
import re
from pg import DB
db = DB()

In [2]:
db.query("""create table mrnas(
         ref_id VARCHAR primary key,
         three_utr VARCHAR,
         shuffled_three_utr VARCHAR
        )""")

In [9]:
ref_seq_to_sequence = dict()
with open("files/3UTR_RefSeq One_record_per_gene") as f:
    firstLine = f.readline()
    ref_seq_id = firstLine.split(" ")[0].split("refGene_")[1]
    mRNA = ""
    for line in f:
        if(line.startswith(">")):
            ref_seq_to_sequence[ref_seq_id] = mRNA
            #add to db if not already exist
#             if(db.query("SELECT * FROM mrnas where ref_id = '" + ref_seq_id + "'").ntuples() == 0):
#                 db.insert('mrnas',ref_id=ref_seq_id, three_utr= mRNA)
            ref_seq_id = line.split(" ")[0].split("refGene_")[1]
            mRNA = ""
        else:
            mRNA += line[0:len(line)-1]
    

In [4]:
q = db.query("SELECT ref_id FROM mrnas")

In [17]:
len(ref_seq_to_sequence)

36658

In [5]:
# ref_seq_to_shuffled_sequence = dict()
window_size = 10
i = 0
for ref_id_t in q.getresult():
    #skip if already defined
    ref_id = ref_id_t[0]
    if(db.query("SELECT ref_id FROM mrnas where ref_id = '" + ref_id +"' and shuffled_three_utr != 'NaN'").ntuples() > 0):
        continue
#     print(ref_id)
#     i += 1
#     if(i > 3):
#         break
    gene = db.query("SELECT three_utr FROM mrnas where ref_id = '" + ref_id + "'").getresult()[0][0]
    window_size = 10
    gene_windows = [gene[i:i+window_size] for i in range(0, len(gene), window_size)]
    shuffled_gene = ""
    for i, geneWindow in enumerate(gene_windows): #shuffling window
        l = list(geneWindow)
        random.shuffle(l)
        shuffled_gene += ''.join(l)
#     print(shuffled_gene)
    do_not_print = db.query("UPDATE mrnas SET shuffled_three_utr = '" + shuffled_gene + "' WHERE ref_id  = '" + ref_id + "'")


#     ref_seq_to_shuffled_sequence[ref_id] = ''.join(gene_windows) 

In [12]:
ref_seq_to_shuffled_sequence

{'NM_032291': 'tgaaatcttatgcaaggatttggaggattcatataatggagaactgatgtatgagaaacagattttaattttggtttgatgaaaacaaaccaatatctgcacttgggatatatcaggtggaaagtcaatgactttcatctgtgatttccctcacacactaccatgatgaccagtcctacagtatttacttctaggtgtaatattgttaatggttttaaaatgtaattattgtatttgtaaattgtactctcattccagtaaggcagttagacacttgagttttagcattttaccattcctgaaatggatgtaatttaaactgtggtatgtaaatttaatagtagtattgttgaatggcacaatgcttacagaggtagattgcattttgtcaatatataaaatttaaatataatattgatagctgtcataaagggggtgccacatattaaagaaacttaagtggaaccagaagaaaaagaaacaaacttacttttcttcaatgcttagtatgttttactctagtgctaaataaaaactctatcttcaaatgtttagtgggttaaattgagaaactatttcagaaaaaaattctaaggttacagcatattcaaagaaaagcattagttaccactttttaaaaagcttttttttcaaactgcaaatttcataaaaatgcaaactgtgtaaacagggcctcttatttttataacttgtgtaaaaagggaaagcaattcatatttaaagtttaagtatattaaattataatcaagagtaaagaagatgttgaagtcttaactacttgcccctctctacagtttcgcaaatgtggggattgctgaataatcagtcagactaaaaccaaaattgtgattttaagatttcaagactttccgtagttgaactggttaagaatttttgcttagttactctgaatagatgatcttactcatccagtatgggggaatgatacctcacgtcttcctct

In [27]:
window_size = 10
geneWindows = [gene[i:i+n] for i in range(0, len(gene), n)]
geneWindows

['gtgggcctcg',
 'ttccccccgg',
 'agaatcccag',
 'cggggcctca',
 'gagatgcatc',
 'tgggagaggt',
 'ggggaagatg',
 'gcaggcaagg',
 'gtacccttgg',
 'ccaggactct',
 'ggtgcccacc',
 'ctgccacccc',
 'cgcgctccac',
 'ctgcagtgtt',
 'tctgccctgt',
 'aaataggacc',
 'agtcttacac',
 'tcgctgtagt',
 'tcaagtatgc',
 'aacataaatc',
 'ctgttccttc',
 'ca']

In [35]:
import random
for i, geneWindow in enumerate(geneWindows): #shuffling window
    l = list(geneWindow)
    random.shuffle(l)
    geneWindows[i] = ''.join(l) 

In [36]:
geneWindows

['gctgctgggc',
 'ctcctcggcc',
 'caatgcacag',
 'ctgggcacgc',
 'atcacggtga',
 'ggggatatgg',
 'ggggggtaaa',
 'gggcggcaaa',
 'gcgtctgtac',
 'cgtcagtcca',
 'cacggcccgt',
 'cccagccctc',
 'acccctgccg',
 'gttagctctg',
 'ccttgcgtct',
 'gacataagca',
 'tctcctaaga',
 'tttcggctga',
 'gtcagatact',
 'cctaaaaata',
 'tctttcgctc',
 'ac']